In [1]:
import os
import numpy as np
import yaml

merge_n_frames = 20

data_path = '/home/shitong/Datasets/04/'
odom_folder = data_path + 'velodyne/'
raw_folder = data_path + 'raw/'
label_folder = data_path + 'labels/'
pose_file = data_path + 'poses.txt'
calib_file = data_path + 'calib.txt'
config_file = './config/semantic-kitti.yaml'

odom_file_list = []
raw_file_list = []
label_file_list = []
'''取所有odom点云文件路径'''
for folder_path, _, file_list in os.walk(odom_folder):
    file_list.sort()
    odom_file_list = [os.path.abspath(os.path.join(folder_path, f)) for f in file_list]
'''取所有raw点云文件路径'''
for folder_path, _, file_list in os.walk(raw_folder):
    file_list.sort()
    raw_file_list = [os.path.abspath(os.path.join(folder_path, f)) for f in file_list]
'''取所有label文件路径'''
for folder_path, _, file_list in os.walk(label_folder):
    file_list.sort()
    label_file_list = [os.path.abspath(os.path.join(folder_path, f)) for f in file_list]
'''读取T_cam_velo'''
with open(calib_file) as file:
    data = file.readlines()
    T_cam_velo34 = data[4].split(':')[1].split(' ')[1:]
    T_cam_velo34 = np.array(T_cam_velo34).astype(np.float32).reshape((3, 4))
    T_cam_velo = np.identity(4)
    T_cam_velo[:3, :] = T_cam_velo34
T_velo_cam = np.linalg.inv(T_cam_velo)
'''读取T_w_cam'''
with open(pose_file) as file:
    data = file.readlines()
    T_w_cam34 = data[0].split(' ')
    T_w_cam34 = np.array(T_w_cam34).astype(np.float32).reshape((3, 4))
    T_w_cam = np.identity(4)
    T_w_cam[:3, :] = T_w_cam34
T_cam_w = np.linalg.inv(T_w_cam)
'''读取pose'''
pose_list = []
with open(pose_file) as file:
    data = file.readlines()
    data = [d.split(' ') for d in data]
    pose_list = [np.array(d).astype(np.float32).reshape((3, 4)) for d in data]
# print(pose_list)
'''读取colormap'''
with open(config_file) as file:
    config = yaml.safe_load(file)
    color_dict = config['color_map']
    color_map = np.zeros((max(color_dict.keys())+ 1, 3))
    for key,value in color_dict.items():
        color_map[key] = np.array(value)

In [2]:
'''读取label'''
color_merged = None
for frame in range(min(merge_n_frames, len(label_file_list))):
    cur_label = np.fromfile(label_file_list[frame], np.int32).reshape(-1)
    sem_label = cur_label & 0xFFFF
    cur_color = color_map[sem_label] / 256.
    # cur_color[:, 0] = 1. - frame / 3 / merge_n_frames
    # cur_color[:, 1] = 1. - frame / 2 / merge_n_frames
    # cur_color[:, 2] = 1. - frame / 4 / merge_n_frames
    if color_merged is None:
        color_merged = cur_color.astype(np.float32)
    else:
        color_merged = np.concatenate([color_merged, cur_color], axis=0).astype(np.float32)
    # print(cur_color)
color_merged = color_merged.astype(np.float32)
color_merged.tofile('././bin/color_merged_pc.color')

In [3]:
odom_merged_pc = None
for frame in range(min(merge_n_frames, len(odom_file_list))):
    # print(odom_file_list[frame])
    cur_pc = np.fromfile(odom_file_list[frame], np.float32).reshape((-1, 4))
    cur_pc_i = cur_pc[:, -1]
    cur_pc[:, -1] = 1.
    cur_pose = np.identity(4)
    cur_pose[:3, :] = pose_list[frame][:3, :]
    cur_pc = (T_velo_cam @ T_cam_w @ cur_pose @ T_cam_velo @ cur_pc.T).T
    cur_pc[:, -1] = cur_pc_i
    if odom_merged_pc is None:
        odom_merged_pc = cur_pc
    else:
        odom_merged_pc = np.concatenate([odom_merged_pc, cur_pc], axis=0)
        
odom_merged_pc = odom_merged_pc.astype(np.float32)
odom_merged_pc.tofile('././bin/odom_merged_pc.bin')

In [4]:
raw_merged_pc = None
for frame in range(min(merge_n_frames, len(raw_file_list))):
    # print(odom_file_list[frame])
    cur_pc = np.fromfile(raw_file_list[frame], np.float32).reshape((-1, 4))
    cur_pc_i = cur_pc[:, -1]
    cur_pc[:, -1] = 1.
    cur_pose = np.identity(4)
    cur_pose[:3, :] = pose_list[frame][:3, :]
    cur_pc = (T_velo_cam @ T_cam_w @ cur_pose @ T_cam_velo @ cur_pc.T).T
    cur_pc[:, -1] = cur_pc_i
    if raw_merged_pc is None:
        raw_merged_pc = cur_pc
    else:
        raw_merged_pc = np.concatenate([raw_merged_pc, cur_pc], axis=0)

raw_merged_pc = raw_merged_pc.astype(np.float32)
raw_merged_pc.tofile('././bin/raw_merged_pc.bin')

In [5]:
import open3d

mesh_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])

odom_pcd = open3d.geometry.PointCloud()
odom_pcd.points = open3d.utility.Vector3dVector(odom_merged_pc[:, :3])
odom_pcd.colors = open3d.utility.Vector3dVector(color_merged)

raw_pcd = open3d.geometry.PointCloud()
raw_pcd.points = open3d.utility.Vector3dVector(raw_merged_pc[:, :3])
raw_pcd.colors = open3d.utility.Vector3dVector(color_merged)

vis_ = open3d.visualization.Visualizer()
vis_.create_window()
vis_.add_geometry(odom_pcd)
# vis_.add_geometry(raw_pcd)
vis_.add_geometry(mesh_frame)
render_options = vis_.get_render_option()
render_options.point_size = 1
render_options.background_color = np.array([0.2, 0.2, 0.2])
vis_.run()
vis_.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
